In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import ipywidgets as widgets
from IPython.display import display, clear_output
import copy

# --- Simulation Constants ---
FIRE_SPREAD_PROBABILITY = 0.25
SMOKE_SPREAD_PROBABILITY = 0.55
MAX_TICKS = 40

# --- 1. Building Layout Generation ---
def create_structured_building(num_rooms=10, hallway_segments=5):
    """
    Generates a graph with a structured layout: a single hallway with rooms.
    """
    G = nx.Graph()
    for i in range(hallway_segments):
        hallway_id = f"H{i}"
        G.add_node(hallway_id, type='hallway', fire=False, smoke=False, length=2)
        if i > 0:
            G.add_edge(f"H{i}", f"H{i-1}")

    for i in range(num_rooms):
        room_id = f"R{i}"
        people = []
        if random.random() > 0.4:
            num_people = random.randint(1, 3)
            for p in range(num_people):
                people.append({'hp': 100})

        G.add_node(room_id, type='room', fire=False, smoke=False, people=people, length=4)
        hallway_connection = f"H{random.randint(0, hallway_segments-1)}"
        G.add_edge(room_id, hallway_connection)

    G.add_node("Exit-1", type='exit', fire=False, smoke=False, length=1)
    G.add_node("Exit-2", type='exit', fire=False, smoke=False, length=1)
    G.add_edge("Exit-1", "H0")
    G.add_edge("Exit-2", f"H{hallway_segments-1}")

    room_nodes = [n for n, d in G.nodes(data=True) if d['type'] == 'room']
    if room_nodes:
        G.graph['fire_start_node'] = random.choice(room_nodes)

    return G

# --- 2. Simulation Logic ---
def update_simulation_step(G):
    """
    Calculates the state for the *next* tick based on the current state.
    """
    next_G = copy.deepcopy(G)
    nodes_on_fire = {n for n, d in G.nodes(data=True) if d['fire']}
    nodes_with_smoke = {n for n, d in G.nodes(data=True) if d['smoke']}

    for node in nodes_on_fire:
        for neighbor in G.neighbors(node):
            if G.nodes[neighbor]['type'] != 'exit' and not G.nodes[neighbor]['fire']:
                if random.random() < FIRE_SPREAD_PROBABILITY:
                    next_G.nodes[neighbor]['fire'] = True

    for node in nodes_on_fire.union(nodes_with_smoke):
        for neighbor in G.neighbors(node):
            if G.nodes[neighbor]['type'] != 'exit' and not G.nodes[neighbor]['smoke']:
                if random.random() < SMOKE_SPREAD_PROBABILITY:
                    next_G.nodes[neighbor]['smoke'] = True

    for node_id, data in next_G.nodes(data=True):
        if data['type'] == 'room' and data['people']:
            for person in data['people']:
                if person['hp'] > 0:
                    if data['fire']:
                        person['hp'] -= 25
                    elif data['smoke']:
                        person['hp'] -= 10
                    person['hp'] = max(0, person['hp'])

    return next_G

def run_full_simulation(initial_layout):
    """
    Runs the entire simulation from start to finish and stores each step.
    """
    history = [initial_layout]
    current_G = initial_layout

    g_tick1 = copy.deepcopy(current_G)
    if 'fire_start_node' in g_tick1.graph:
        start_node = g_tick1.graph['fire_start_node']
        g_tick1.nodes[start_node]['fire'] = True
    history.append(g_tick1)
    current_G = g_tick1

    for _ in range(2, MAX_TICKS + 1):
        next_G = update_simulation_step(current_G)
        history.append(next_G)
        current_G = next_G

    return history

# --- 3. Visualization ---
def draw_graph(sim_graph, tick):
    """
    Helper function to draw a single graph state.
    """
    pos = nx.spring_layout(sim_graph, seed=42, iterations=100)

    node_colors = []
    for node in sim_graph.nodes():
        if sim_graph.nodes[node]['fire']:
            node_colors.append('red')
        elif sim_graph.nodes[node]['smoke']:
            node_colors.append('darkgrey')
        elif sim_graph.nodes[node]['type'] == 'exit':
            node_colors.append('lime')
        elif sim_graph.nodes[node]['type'] == 'room':
            node_colors.append('skyblue')
        else:
            node_colors.append('khaki')

    labels = {}
    for node_id, data in sim_graph.nodes(data=True):
        label = f"{node_id}\n(L: {data['length']})"
        if data['type'] == 'room' and data['people']:
            people_hp = [f"HP:{p['hp']}" for p in data['people']]
            label += f"\n{', '.join(people_hp)}"
        labels[node_id] = label

    plt.figure(figsize=(18, 12))
    nx.draw(sim_graph, pos, with_labels=True, labels=labels, node_color=node_colors,
            node_size=3500, font_size=9, font_weight='bold')

    if tick == 0:
        plt.title("Initial State - Tick: 0", fontsize=20)
    else:
        plt.title(f"Current State - Tick: {tick}", fontsize=20)

    plt.show()

def visualize_comparison(tick):
    """
    Visualizes the current tick and, if applicable, the initial state (Tick 0).
    """
    global simulation_history
    if not simulation_history or tick >= len(simulation_history):
        # This check prevents errors when the widget is still loading
        return

    # 1. Draw the map for the currently selected tick
    current_graph = simulation_history[tick]
    draw_graph(current_graph, tick)

    # 2. If we are not at the start, also draw the initial map for comparison
    if tick > 0:
        initial_graph = simulation_history[0]
        draw_graph(initial_graph, 0)

# --- 4. Main Execution and Interactive Widgets ---
simulation_history = []

time_slider = widgets.IntSlider(
    value=0, min=0, max=MAX_TICKS, step=1,
    description='Time (Ticks):', continuous_update=False, layout={'width': '60%'})

run_button = widgets.Button(description="Run New Simulation", button_style='success')
output_area = widgets.Output()

def on_run_button_clicked(b):
    global simulation_history
    with output_area:
        clear_output(wait=True)
        initial_layout = create_structured_building(num_rooms=12, hallway_segments=6)
        simulation_history = run_full_simulation(initial_layout)

        # Resetting the slider's value will automatically trigger visualize_comparison(0)
        # No need to call it manually. This was the source of the extra plot.
        time_slider.value = 0

run_button.on_click(on_run_button_clicked)

# Link the slider to the new comparison visualization function
interactive_plot = widgets.interactive_output(visualize_comparison, {'tick': time_slider})

# Display the final layout
print("--- Simulation Controls ---")
display(run_button)
display(time_slider)
display(output_area)
display(interactive_plot)

# Run an initial simulation on startup
on_run_button_clicked(None)

--- Simulation Controls ---


Button(button_style='success', description='Run New Simulation', style=ButtonStyle())

IntSlider(value=0, continuous_update=False, description='Time (Ticks):', layout=Layout(width='60%'), max=40)

Output()

Output()